In [50]:
import os
import sys
import glob

import numpy as np
import scipy as sp
import pandas as pd
import sklearn as sk
from sklearn.cross_validation import train_test_split
from sklearn import tree
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import ggplot

from IPython.display import HTML

%matplotlib inline

In [18]:
DATA_PATH = "/Users/beingzy/Documents/Projects/learning_dist_metrics_exp"
RANDOM_SEED = 123

df = pd.read_csv(DATA_PATH + "/udiff_df.csv", header = 0)
print df.head(5)

feat_columns = [col for col in df.columns if not col in ["class", "uid_a", "uid_b"]]
print feat_columns

   class  uid_a  uid_b  birthday  education_classes_id  \
0      1    236    186       NaN                   NaN   
1      1    122    285       NaN                   NaN   
2      1     24    346       NaN                   NaN   
3      1    271    304       NaN                   NaN   
4      1    176      9       NaN                   NaN   

   education_concentration_id  education_degree_id  education_school_id  \
0                         NaN                  NaN                  NaN   
1                         NaN                  NaN                    0   
2                           5                  NaN                    0   
3                         NaN                  NaN                  NaN   
4                         NaN                  NaN                  NaN   

   education_type  education_with_id      ...       political  religion  \
0             NaN                NaN      ...             NaN       NaN   
1               2                NaN      ...     

In [4]:
def summarize_null_ptg(ds):
    """Return the percentage of obs. with missing value
       per column
       
       Parameters:
       ----------
       ds: {matrix, pd.DataFrame}
    """
    na_ptg = dict()
    for col in ds.columns:
        na_ptg[col] = ds[col].isnull().sum() * 1.0 / df.shape[0]
    return na_ptg


print summarize_null_ptg(df)
print "#obs: {0}, #feats: {1}".format(df.shape[0], df.shape[1])

{'last_name': 0.82320836157385935, 'locale': 0.29243659258730781, 'education_type': 0.33015107195091498, 'hometown_id': 0.85051546391752575, 'education_concentration_id': 0.8773543680662077, 'location_id': 0.77205436449898335, 'first_name': 0.99290122355794952, 'work_employer_id': 0.96290532586594368, 'middle_name': 0.99885848821032353, 'education_classes_id': 0.99754307423393862, 'political': 1.0, 'work_with_id': 0.99997324581742941, 'work_position_id': 0.98398316270110231, 'religion': 1.0, 'education_with_id': 0.99963435950486923, 'work_location_id': 0.97319676809474542, 'work_end_date': 0.94297345985088998, 'uid_b': 0.0, 'uid_a': 0.0, 'work_start_date': 0.90594121214283163, 'education_school_id': 0.38457353832982555, 'birthday': 0.75150715228480724, 'work_projects_id': 0.99908143973174468, 'class': 0.0, 'name': 1.0, 'gender': 0.29489351835336924, 'education_year_id': 0.56773713123818359, 'work_from_id': 0.99998216387828631, 'education_degree_id': 0.98419719616166657, 'languages_id':

In [20]:
## Missing value handling:
is_feat_na = df[feat_columns].isnull().sum(axis = 1) == len(feat_columns)
df_reduced = df[ ~is_feat_na ] #
print "#obs: {0}, #feats: {1}".format(df_reduced.shape[0], df_reduced.shape[1])
## count the distribution of {0, 1}
print df_reduced["class"].value_counts()

#obs: 162054, #feats: 30
1    110366
0     51688
dtype: int64


In [21]:
df_x = df_reduced[feat_columns].fillna(0)
df_y = df_reduced["class"]

train_x, test_x, train_y, test_y = \
    train_test_split(df_x, df_y, test_size = .2, random_state = RANDOM_SEED)

print "#obs (training): {0}, #obs (testing): {1}".format(train_x.shape[0], test_x.shape[0])

#obs (training): 129643, #obs (testing): 32411


In [7]:
clf = tree.DecisionTreeClassifier()
clf.fit(train_x, train_y)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=None, splitter='best')

In [10]:
from sklearn.externals.six import StringIO  
import pydot 
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
#graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
#graph.write_pdf("iris.pdf") 

In [11]:
from sklearn import metrics
def measure_performance(X,y,clf, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True):
    y_pred=clf.predict(X)   
    if show_accuracy:
        print "Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred)),"\n"

    if show_classification_report:
        print "Classification report"
        print metrics.classification_report(y,y_pred),"\n"
        
    if show_confusion_matrix:
        print "Confusion matrix"
        print metrics.confusion_matrix(y,y_pred),"\n"
        
measure_performance(train_x, train_y, clf, show_classification_report=True, show_confusion_matrix=True)

Accuracy:0.951 

Classification report
             precision    recall  f1-score   support

          0       0.97      0.88      0.92     41385
          1       0.94      0.99      0.96     88258

avg / total       0.95      0.95      0.95    129643


Confusion matrix
[[36280  5105]
 [ 1218 87040]] 



In [12]:
measure_performance(test_x, test_y, clf, show_classification_report=True, show_confusion_matrix=True)

Accuracy:0.875 

Classification report
             precision    recall  f1-score   support

          0       0.87      0.71      0.78     10303
          1       0.87      0.95      0.91     22108

avg / total       0.87      0.87      0.87     32411


Confusion matrix
[[ 7289  3014]
 [ 1048 21060]] 



In [16]:
from sklearn import pipeline
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import auc

#clf_pipeline = pipeline([( 'clf_tree', tree.DecisionTreeClassifier() )])
clf_tree = tree.DecisionTreeClassifier()
param_grids = [
    {"clf_tree__criterion": ["gini"]},
    #{"clf_tree__criterion": ["gini"], "clf_tree__splitter": ["best"], "clf_tree__min_samples_leaf": [10, 20, 50, 100]},
    #{"clf_tree__criterion": ["entropy"], "clf_tree__splitter": ["best"], "clf_tree__min_samples_leaf": [10, 20, 50, 100]}
    ]
score_func =  [
    ('precision', precision_score),
    ('recall', recall_score),
]

grid_ridge = GridSearchCV(clf_tree, 
                          param_grid = param_grids, 
                          refit = True,
                          n_jobs = -1, 
                          scoring = score_func,
                          cv = StratifiedKFold(train_y, n_folds=5))
perf_report = grid_ridge.fit(train_x, train_y)

An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (2, 0))
An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (2, 0))
An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (2, 0))





JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/Applications/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='IPython.kernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/Applications/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'IPython.kernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

............

In [26]:
print feat_columns

['birthday', 'education_classes_id', 'education_concentration_id', 'education_degree_id', 'education_school_id', 'education_type', 'education_with_id', 'education_year_id', 'first_name', 'gender', 'hometown_id', 'languages_id', 'last_name', 'locale', 'location_id', 'middle_name', 'name', 'political', 'religion', 'work_employer_id', 'work_end_date', 'work_from_id', 'work_location_id', 'work_position_id', 'work_projects_id', 'work_start_date', 'work_with_id']


In [30]:
df.describe()

,class,uid_a,uid_b,birthday,education_classes_id,education_concentration_id,education_degree_id,education_school_id,education_type,education_with_id,...,political,religion,work_employer_id,work_end_date,work_from_id,work_location_id,work_position_id,work_projects_id,work_start_date,work_with_id
count,224264.000000,224264.000000,224264.000000,55728.000000,551.000000,27505.000000,3544.000000,138018.000000,150223.000000,82.000000,...,0,0,8319.000000,12789.000000,4,6011.000000,3592.000000,206.000000,21094.000000,6
mean,0.641414,1688.622414,1772.142894,87.816789,229.125227,185.561643,141.008183,161.600400,0.262550,9.280488,...,NaN,NaN,219.593341,74.976777,0,234.330727,286.505846,6.393204,150.509529,0
std,0.479586,831.244742,810.630781,191.001289,338.094119,211.882213,180.172678,229.809663,0.618928,62.462487,...,NaN,NaN,286.476229,174.076806,0,288.406133,327.466048,28.123868,224.860662,0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0
25%,0.000000,1118.000000,1102.000000,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,5.000000,2.000000,0,0.000000,3.000000,1.000000,7.000000,0
50%,1.000000,1737.000000,1931.000000,1.000000,8.000000,78.000000,5.000000,31.000000,0.000000,0.000000,...,NaN,NaN,85.000000,9.000000,0,51.000000,11.500000,4.000000,36.000000,0
75%,1.000000,2340.000000,2410.000000,5.000000,733.000000,377.000000,202.000000,289.000000,0.000000,1.000000,...,NaN,NaN,404.000000,28.000000,0,479.000000,513.000000,5.000000,153.000000,0
max,1.000000,3213.000000,3213.000000,1006.000000,747.000000,1001.000000,756.000000,1153.000000,2.000000,536.000000,...,NaN,NaN,1114.000000,819.000000,0,1060.000000,986.000000,289.000000,1014.000000,0


#### Variale Processing
* 

In [29]:
remove_feats = ['birthday', 'education_classes_id', 
                
def feature_processor(dataset):
    """
    """
ds = dataset.copy()
    #ds["birthday"] 
ds["education_classes_id"] = df["education_classes_id"].apply(lambda x: 1 if x > 0 else 0)
ds["education_concentration_id"] = df["education_concentration_id"].apply(lambda x: 1 if x > 0 else 0)
    #ds["education_degree_id"] 
ds["education_school_id"] = df["education_schoold_id"].apply(lambda x: 1 if x > 0 else 0)
ds["education_type_id"] = df["education_type_id"].apply(lambda x: 1 if x > 0 else 0)
ds["
                
        
#feat_columns.index("education_classes_id")

1

#### Clustering Analysis To Detect Underlying Pattern of The Data
* K-means Clustering

In [22]:
km_cluster = KMeans()
km_cluster.fit(train_x)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=8, n_init=10,
    n_jobs=1, precompute_distances=True, random_state=None, tol=0.0001,
    verbose=0)

In [23]:
tran_class = km_cluster.predict(train_x)

In [24]:
km_cluster.inertia_

12801191981.687754

In [48]:
#df["education_classes_id"][:100].apply(lambda x: 1 if x > 0 else 0)
df["education_concentration_id"].unique

<bound method Series.unique of 0    NaN
1    NaN
2      5
3    NaN
4    NaN
5    NaN
6      0
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14     0
...
224249    NaN
224250    NaN
224251    NaN
224252    NaN
224253    NaN
224254    NaN
224255    185
224256    NaN
224257    198
224258    NaN
224259    NaN
224260    NaN
224261    403
224262    NaN
224263    NaN
Name: education_concentration_id, Length: 224264, dtype: float64>

In [44]:
df["education_classes_id"].unique()

array([  nan,    3.,    1.,    0.,    2.,    4.,  746.,    5.,    8.,
         14.,  736.,  739.,   10.,  733.,   13.,  745.,  747.,    7.,
          6.,   11.,    9.,  732.,  738.,   15.,  741.,  731.,   16.,
         12.,  734.,  735.,  325.,  744.,  737.,  414.])